In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import datatable as dt
from numpy import concatenate
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from scipy import stats
from scipy.stats import norm, skew 
import matplotlib as mpl
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e6/sample_submission.csv
/kaggle/input/playground-series-s3e6/train.csv
/kaggle/input/playground-series-s3e6/test.csv
/kaggle/input/paris-housing-price-prediction/ParisHousing.csv


In [2]:
train = pd.read_csv('../input/playground-series-s3e6/train.csv', index_col=0);
train.head()

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
id,,,,,,,,,,,,,,,,,
0,34291,24,1,0,47,35693,2,1,2000,0,1,8,5196,369,0,3,3436795.2
1,95145,60,0,1,60,34773,1,4,2000,0,1,729,4496,277,0,6,9519958.0
2,92661,45,1,1,62,45457,4,8,2020,1,1,7473,8953,245,1,9,9276448.1
3,97184,99,0,0,59,15113,1,1,2000,0,1,6424,8522,256,1,9,9725732.2
4,61752,100,0,0,57,64245,8,4,2018,1,0,7151,2786,863,0,7,6181908.8


In [3]:
test = pd.read_csv('../input/playground-series-s3e6/test.csv', index_col=0);
test.head()

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom
id,,,,,,,,,,,,,,,,
22730,47580,89,0,1,8,54830,5,3,1995,0,0,6885,8181,241,0,8
22731,62083,38,0,0,87,8576,10,3,1994,1,1,4601,9237,393,1,4
22732,90499,75,1,1,37,62454,9,6,1997,0,1,7454,2680,305,0,2
22733,16354,47,1,1,9,9262,6,5,2019,1,1,705,5097,122,1,5
22734,67510,8,0,0,55,24112,3,7,2014,1,1,3715,7979,401,1,9


In [4]:
extend_df = pd.read_csv('../input/paris-housing-price-prediction/ParisHousing.csv');
extend_df.head()

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
0,75523,3,0,1,63,9373,3,8,2005,0,1,4313,9005,956,0,7,7559081.5
1,80771,39,1,1,98,39381,8,6,2015,1,0,3653,2436,128,1,2,8085989.5
2,55712,58,0,1,19,34457,6,8,2021,0,0,2937,8852,135,1,9,5574642.1
3,32316,47,0,0,6,27939,10,4,2012,0,1,659,7141,359,0,3,3232561.2
4,70429,19,1,1,90,38045,3,7,1990,1,0,8435,2429,292,1,4,7055052.0


In [5]:
r1,c1 = train.shape
print('The training data has {} rows and {} columns'.format(r1,c1))
r2,c2 = test.shape
print('The test data has {} rows and {} columns'.format(r2,c2))
r3,c3 = extend_df.shape
print('The original data has {} rows and {} columns'.format(r3,c3))

The training data has 22730 rows and 17 columns
The test data has 15154 rows and 16 columns
The original data has 10000 rows and 17 columns


In [6]:
# To see the quantity of null vaues in all the columns.
# c1 stands for the number of columns in the training data.

print('MISSING VALUES IN TRAINING DATASET:')
print(train.isna().sum().nlargest(c1))
print('')
print('MISSING VALUES IN TEST DATASET:')
print(test.isna().sum().nlargest(c2))
print('')
print('MISSING VALUES IN ORIGINAL DATASET:')
print(extend_df.isna().sum().nlargest(c3))

MISSING VALUES IN TRAINING DATASET:
squareMeters         0
isNewBuilt           0
hasGuestRoom         0
hasStorageRoom       0
garage               0
attic                0
basement             0
hasStormProtector    0
made                 0
numberOfRooms        0
numPrevOwners        0
cityPartRange        0
cityCode             0
floors               0
hasPool              0
hasYard              0
price                0
dtype: int64

MISSING VALUES IN TEST DATASET:
squareMeters         0
numberOfRooms        0
hasYard              0
hasPool              0
floors               0
cityCode             0
cityPartRange        0
numPrevOwners        0
made                 0
isNewBuilt           0
hasStormProtector    0
basement             0
attic                0
garage               0
hasStorageRoom       0
hasGuestRoom         0
dtype: int64

MISSING VALUES IN ORIGINAL DATASET:
squareMeters         0
isNewBuilt           0
hasGuestRoom         0
hasStorageRoom       0
garage           

In [7]:
train = pd.concat([train, extend_df]); 
train['made'] = 2021 - train['made']
train.head()

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
0,34291,24,1,0,47,35693,2,1,21,0,1,8,5196,369,0,3,3436795.2
1,95145,60,0,1,60,34773,1,4,21,0,1,729,4496,277,0,6,9519958.0
2,92661,45,1,1,62,45457,4,8,1,1,1,7473,8953,245,1,9,9276448.1
3,97184,99,0,0,59,15113,1,1,21,0,1,6424,8522,256,1,9,9725732.2
4,61752,100,0,0,57,64245,8,4,3,1,0,7151,2786,863,0,7,6181908.8


In [8]:
r1_2,c1_2 = train.shape
print('The training data has {} rows and {} columns'.format(r1_2,c1_2))

The training data has 32730 rows and 17 columns


In [9]:
features = ['squareMeters','numberOfRooms','hasYard','hasPool','cityPartRange','cityCode','floors','numPrevOwners','made','isNewBuilt',
            'hasStormProtector','basement','attic','garage','hasStorageRoom','hasGuestRoom']
y = train['price']
X_train,X_test, y_train,y_test = train_test_split(train[features],train['price'],test_size=0.25)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((24547, 16), (8183, 16), (24547,), (8183,))

In [10]:
test['made'] = 2021 - test['made']
test.head()

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom
id,,,,,,,,,,,,,,,,
22730,47580,89,0,1,8,54830,5,3,26,0,0,6885,8181,241,0,8
22731,62083,38,0,0,87,8576,10,3,27,1,1,4601,9237,393,1,4
22732,90499,75,1,1,37,62454,9,6,24,0,1,7454,2680,305,0,2
22733,16354,47,1,1,9,9262,6,5,2,1,1,705,5097,122,1,5
22734,67510,8,0,0,55,24112,3,7,7,1,1,3715,7979,401,1,9


In [11]:
r2_2,c2_2 = test.shape
print('The test data has {} rows and {} columns'.format(r2_2,c2_2))

The test data has 15154 rows and 16 columns


In [12]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score, KFold
import math

MODEL_PARAMS = {       
    'booster': 'gbtree',
    'learning_rate': 0.11,
    'n_estimators': 77,
    'objective': 'reg:squarederror',
    'gamma': 1,
    'max_depth': 4,
    'reg_lambda': 1,
    'reg_alpha':1,
    'subsample': 0.85,
    'colsample_bytree': 1,
    'min_child_weight': 2,
    'seed': 42
}

xgbr = xgb.XGBRegressor(**MODEL_PARAMS)
xgbr.fit(X_train, y_train)
 
kfold = KFold(n_splits=10, shuffle=True)
kf_cv_scores = cross_val_score(xgbr, X_train, y_train, cv=kfold )
print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

ypred2 = xgbr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, ypred2))
print("RMSE: %.5f" % rmse)

K-fold CV average score: 1.00
RMSE: 81935.79940


In [13]:
test_preds = xgbr.predict(test)

#Check the form of submission 
submission_df = pd.read_csv('../input/playground-series-s3e6/sample_submission.csv')
submission_df

,id,price
0,22730,4634456.897
1,22731,4634456.897
2,22732,4634456.897
3,22733,4634456.897
4,22734,4634456.897
...,...,...
15149,37879,4634456.897
15150,37880,4634456.897
15151,37881,4634456.897
15152,37882,4634456.897


In [14]:
test_preds #one array
submission_df['price'] = test_preds
submission_df.to_csv('submission.csv', index=False)

In [15]:
test_preds 

array([4767688.5, 6201282. , 9041574. , ..., 7374080.5, 7773335. ,
       1766437.8], dtype=float32)

In [16]:
# Solution table in the dataset
solution = pd.read_csv('submission.csv')
solution

,id,price
0,22730,4767688.5
1,22731,6201282.0
2,22732,9041574.0
3,22733,1638323.1
4,22734,6753539.5
...,...,...
15149,37879,8607062.0
15150,37880,4572489.0
15151,37881,7374080.5
15152,37882,7773335.0
